In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! git clone https://github.com/big-data-lab-umbc/cybertraining.git

In [ ]:
!ls

'Copy of CCM Project Proposal.gdoc'   sea_ice.csv
 cybertraining			      Synthetic_data_gen.ipynb
 Granger_Causality_Test.ipynb	      Team_Project_Causal_AI.ipynb
 Resources			      Untitled1.ipynb
 sea_ice1.csv


In [ ]:
%cd /content/drive/MyDrive/Fall_2022/IS_800_Causality_New/Team Project2

/content/drive/MyDrive/Fall_2022/IS_800_Causality_New/Team Project


In [ ]:
import numpy as np
import os

# File inputs
#path='/content/drive/MyDrive/Fall_2022/IS_800_Causality_New/Team Project'

path="/content/drive/MyDrive/Fall_2022/IS_800_Causality_New/Team Project/cybertraining/year-2-projects/team-2/EOF_Data/"
f=sorted([f for f in os.listdir(path) if f.startswith('eof_ts')],key=str.lower)
x=[None]*len(f)
for i in np.arange(0,len(f)):
    print(f[i][7:])
    x[i]=np.loadtxt(path+f[i])

msl-00
msl-01
msl-02
msl-03
msl-04
msl-05
msl-06
msl-07
msl-08
msl-09
msl-10
msl-11
msl-12
msl-13
msl-14
msl-15
msl-16
msl-17
msl-18
msl-19
msl-20
msl-21
msl-22
msl-23
msl-24
msl-25
msl-26
msl-27
msl-28
msl-29
si10-00
si10-01
si10-02
si10-03
si10-04
si10-05
si10-06
si10-07
si10-08
si10-09
si10-10
si10-11
si10-12
si10-13
si10-14
si10-15
si10-16
si10-17
si10-18
si10-19
si10-20
si10-21
si10-22
si10-23
si10-24
si10-25
si10-26
si10-27
si10-28
si10-29
sst-00
sst-01
sst-02
sst-03
sst-04
sst-05
sst-06
sst-07
sst-08
sst-09
sst-10
sst-11
sst-12
sst-13
sst-14
sst-15
sst-16
sst-17
sst-18
sst-19
sst-20
sst-21
sst-22
sst-23
sst-24
sst-25
sst-26
sst-27
sst-28
sst-29
t2m-00
t2m-01
t2m-02
t2m-03
t2m-04
t2m-05
t2m-06
t2m-07
t2m-08
t2m-09
t2m-10
t2m-11
t2m-12
t2m-13
t2m-14
t2m-15
t2m-16
t2m-17
t2m-18
t2m-19
t2m-20
t2m-21
t2m-22
t2m-23
t2m-24
t2m-25
t2m-26
t2m-27
t2m-28
t2m-29


In [ ]:
# Hyperparameters for phase space reconstruction and CCM
rlag=1
rdim=3
maxdelay=3

# Parameters for phase space reconstruction and CCM
length=x[0].shape[0]
reconst_length=length-rlag*(rdim-1)
reconst_range=np.arange(rlag*(rdim-1),length)
delay=np.arange(-maxdelay,maxdelay+1)

# The library for cross mapping is a little shorter
lib=np.arange(max(rlag*(rdim-1),maxdelay),length-maxdelay)

print('length='+str(length))
print('reconstucted length='+str(reconst_length))
print('reconstucted range: '+str(reconst_range[0])+' ~ '+str(reconst_range[-1]))
print('library: '+str(lib[0])+' ~ '+str(lib[-1]))
print('CCM delay: '+str(delay[0])+' ~ '+str(delay[-1]))

length=480
reconstucted length=478
reconstucted range: 2 ~ 479
library: 3 ~ 476
CCM delay: -3 ~ 3


In [ ]:
import sys
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr
import csv

# Write outputs to a .csv file
with open('ccm_python.csv',mode='w') as output_file:
    ccm_writer=csv.writer(output_file,delimiter=',')
    ccm_writer.writerow(['Effect','Cause','Timelag','Strength','Method'])

    # CCM computation
    for j in np.arange(0,len(f)):
    
        # Phase space reconstruction for the effect variable x[j]
        reconst=np.zeros([reconst_length,rdim])
        for d in np.arange(0,rdim):
            reconst[:,d]=x[j][d*rlag:d*rlag+reconst_length]

        # Compute Euclidean distances in the reconstructed phase space
        dist=cdist(reconst[lib-rlag*(rdim-1),:],reconst[lib-rlag*(rdim-1),:])
    
        # Exclude the pair of identical phase points
        for k in np.arange(0,len(lib)):
            dist[k,k]=sys.maxsize
                
        # Find the rdim+1 nearest neighbors and assign weights
        index=np.argsort(dist,axis=0)
        sort_dist=np.sort(dist,axis=0)
        weight=np.exp(-sort_dist[:rdim+1]/sort_dist[0])
    
        for i in np.arange(0,len(f)):
        
            # The first dimension is {correlation coefficient, p-value};
            # the second dimension is delay.
            cmskill=np.zeros([2,len(delay)])
            for l in np.arange(0,len(delay)):
                estimate=np.average(x[i][lib[index[:rdim+1]]-delay[l]],axis=0,weights=weight)
                cmskill[:,l]=pearsonr(estimate,x[i][lib-delay[l]])
                
            # Find the optimal delay and corresponding CCM skill
            idx=np.argsort(cmskill[1,:])
            optdelay=delay[idx[0]]
            p=cmskill[1,idx[0]]
            
            # Only preserve meaningful and significant results
            if optdelay>0 and p<0.05:
                ccm_writer.writerow([f[j][7:],f[i][7:],str(optdelay),str(p),'CCM'])
 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Fall_2022/IS_800_Causality_New/Team Project/ccm_python.csv")
df

,Effect,Cause,Timelag,Strength,Method
0,msl-00,msl-01,1,1.171308e-22,CCM
1,msl-00,msl-02,2,2.256661e-42,CCM
2,msl-00,msl-08,1,4.295192e-52,CCM
3,msl-00,msl-09,1,6.543335e-14,CCM
4,msl-00,msl-10,1,3.277078e-45,CCM
...,...,...,...,...,...
5498,t2m-29,t2m-16,2,5.292272e-04,CCM
5499,t2m-29,t2m-20,2,1.791803e-07,CCM
5500,t2m-29,t2m-23,2,1.500256e-02,CCM
5501,t2m-29,t2m-24,1,1.507985e-03,CCM
